In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
test = pd.read_csv("../input/santander-customer-transaction-prediction-dataset/test.csv")
train = pd.read_csv("../input/santander-customer-transaction-prediction-dataset/train.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In this dataset, an anonymous dataset containing numeric variables. In which train contains:ID_code (string),
target and test contains:ID_code (string).

In [ ]:
train.shape,test.shape

Two dataframes have 200000 data,train has 202 columns and test has 201 columns.

In [ ]:
train.dtypes.head()

In [ ]:
test.dtypes.head()

In [ ]:
test.isna().sum(), 

In [ ]:
train.isna().sum()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.nunique()

In [ ]:
test.nunique()

In [ ]:
train.target.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(train.target)

We can see that the data is unbalanced. Majority values have 0 as target value.

Now we know we have a highly imbalanced dataset; 10% are 'buy' transactions, 90% are 'not buy' transactions.

In [ ]:
train.std(axis = 0, skipna = True) 

In [ ]:
train.corr()

In [ ]:
test.corr()

In [ ]:
features = train.columns.values[2:102]
plt.figure(figsize=(20,10))
plt.title("Distribution of mean values per column in the train and test set")
sns.distplot(train[features].mean(axis=0),color="black",kde=True,bins=120, label='train')
sns.distplot(test[features].mean(axis=0),color="blue", kde=True,bins=120, label='test')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.title("Distribution of std values per column in the train and test set")
sns.distplot(train[features].std(axis=0),color="black",kde=True,bins=120, label='train')
sns.distplot(test[features].std(axis=0),color="red", kde=True,bins=120, label='test')
plt.legend()
plt.show()

Distribution of the std, mean value per line in the train dataset, grouped by the value of the target.

In [ ]:
correlations = train[features].corr().abs().unstack().sort_values(kind="quicksort").reset_index()
correlations = correlations[correlations['level_0'] != correlations['level_1']]
correlations.head(10)

In [ ]:
correlations.tail(10)

**Matrix of features by splitting the dependent and independent variables¶


In [ ]:
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values

In [ ]:
y = train.target

In [ ]:
#independent variables
X

In [ ]:
#dependent variables column
y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

> Logistic Regression

In [ ]:
labelEncoderY = LabelEncoder()
y = labelEncoderY.fit_transform(y)

In [ ]:
X = train.drop(['target', 'ID_code'],1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
Y_test = test.drop(columns = ['ID_code'])

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

In [ ]:
sm = SMOTE(random_state=42)
X_train_re, y_train_re = sm.fit_sample(X_train, y_train.ravel())

In [ ]:
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_train_re, y_train_re, test_size = 0.2, random_state = 123, stratify = y_train_re)

In [ ]:
from sklearn.linear_model import LogisticRegression
lrg = LogisticRegression()

lrg.fit(X_train_re, y_train_re)

y_pred_lrg = lrg.predict(X_train_re)
print(classification_report(y_train_re, y_pred_lrg))

In [ ]:
lrg.score(X_train_re, y_train_re)

In [ ]:
accuracy_score(y_train_re, y_pred_lrg)

In [ ]:
logist_pred = lrg.predict_proba(X_test)[:,1]

In [ ]:
logist_pred_test = lrg.predict_proba(Y_test)[:,1]
submit = test[['ID_code']]
submit['target'] = logist_pred_test
submit.head()

In [ ]:
submit.to_csv('log_reg_baseline.csv', index = False)

Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [ ]:
gnb.fit(X_train_re, y_train_re)

In [ ]:
y_pred_gnb = gnb.predict(X_train_re)

In [ ]:
recall_score(y_train_re, y_pred_gnb)

In [ ]:
gnb_pred_test = gnb.predict_proba(Y_test)[:,1]
submit = test[['ID_code']]
submit['target'] = gnb_pred_test
submit.head()

In [ ]:
submit.to_csv('NB_baseline.csv', index = False)

In [ ]:
##Support Vector Machines (SVM)
##from sklearn.svm import SVC
##svm = SVC(kernel = 'linear')
##svm.fit(X_train_re, y_train_re)
##y_pred_svm = svm.predict(X_train_re)
##recall_score(y_train_re, y_pred_svm)

In [ ]:
##RandomForestClassification
##from sklearn.ensemble import RandomForestClassifier
##rf = RandomForestClassifier(n_estimators=100)
##rf.fit(X_train, y_train)


In [ ]:
##MuLtilayer Perceptron (MLP)
##from sklearn.neural_network import MLPClassifier
##mlp = MLPClassifier()
##mlp.fit(X_train_re, y_train_re)
##y_pred_mlp = svm.predict(X_test)